In [4]:
import pandas as pd
import os
import xml.etree.ElementTree as ET ## XML parsing
from collections import defaultdict
import numpy as np
from itunesLibrary import library

from itunes_lib_scan import LibScan
import pickle 

import dateparser
        
import music_tag

%load_ext autoreload
%autoreload 2

In [47]:
lib_name = '/Users/antoine/Developer/itunes-lib-dat/iTunes-Library-2024-03-10'
sd_walkman_path = '/Volumes/SD_CARD/MUSIC'
path_to_music_folder = '/Volumes/MasterAudio/Audio/Gros iTunes/'

create_cover_jpg = False
create_album_jpg = False
complete_missing_cover_art = False
convert_to_non_prog = False
force_scan = False
skip_scanning = False


LS = LibScan(path_to_library_file=lib_name,
             path_to_music_folder = path_to_music_folder,
             create_cover_jpg = create_cover_jpg,
             create_album_jpg = create_album_jpg,
             complete_missing_cover_art = complete_missing_cover_art,
             convert_to_non_prog = convert_to_non_prog,
             force_scan = force_scan,
             skip_scanning = skip_scanning)




! Be Patient this will take a few seconds !
 ! Warning this part is going to modify your music files !
! Be Patient. Scanning all the music your /Volumes/MasterAudio/Audio/Gros iTunes/ will take a long time (~15mins for a 30k songs lib with a Mac mini M2) !
**** Album wo artwork:  /Volumes/MasterAudio/Audio/Gros iTunes/2000-2020-Singles/Simples Vf/Le meilleur de la chanson française, vol. 3 (Remasterisé)


In [44]:
LS.get_all_playlists_from_a_lib()
LS.get_all_playlists_with_alternative_path(new_root_folder = sd_walkman_path)
LS.get_locations_with_multiple_album()

Sp-Tintin.m3u8
ML-Piano.m3u8
2020-Confinement.m3u8
BS-Gould-The Well-Tempered Clavier.m3u8
BS-Chopin.m3u8
2021 iPodsU2.m3u8
Cpl-MagTechnoA.m3u8
AL-*** PodBlanc.m3u8
BS-BillEvans.m3u8
lowres 191.m3u8
*.m3u8
MD-JL Guerra.m3u8
**.m3u8
AL-Pais.m3u8
2004-Papa Mozart.m3u8
G-House80's.m3u8
AL-JazzPiano.m3u8
AL-Traxx.m3u8
SW-Enfant.m3u8
Les Aventures de Tintin.m3u8
Cpl-ClassicHouse***.m3u8
2005-10 Electro.m3u8
G-Rock80's.m3u8
MD-JazzVocal.m3u8
AL-PianoChambre.m3u8
Compiles.m3u8
AL-Barber.m3u8
MD-IbizaLounge.m3u8
2003-MP3 Mix.m3u8
2005-10 Divers.m3u8
Cpl-24H Nova 2.m3u8
Sans Note.m3u8
2005-10 PopRock.m3u8
Playlist 2.m3u8
MD-Louis Armstrong.m3u8
2010-15 Divers.m3u8
MD-Noël.m3u8
G-Rock90's.m3u8
Playlist 3.m3u8
BS-DomingoPavarotti.m3u8
MD-BossaNova.m3u8
G-TechnoDetroit.m3u8
MD-GuitareClassique.m3u8
DankPods.m3u8
2003-MP3 Compiles.m3u8
MD-ElectroLatino.m3u8
Cpl-iPod.m3u8
****.m3u8
Sherlock.m3u8
2004-Papa Divers.m3u8
2021 iPodsAlbums.m3u8
AL-AlaSerna 1.m3u8
MD-Tchaikovsky.m3u8
2015-20 Classique.m3u8

KeyError: 'Album Location'

In [2]:


audio_locations = ['/Volumes/MasterAudio/Audio/']
levels = [0]

df = get_lib_as_csv(lib_name = lib_name)

##########################
# EDIT THE 5 LINES BELOW #
##########################

split_lib(lib_name,
            split_type = ['Genre','Album Scoring',],
              maximum_size_feature = 'Album Size' ,
              maximum_bin_size_in_hours = 4,
              maximum_bin_size_in_gb = 10)

split_lib(lib_name,
            split_type = ['Genre','Album Scoring',],
              maximum_size_feature = 'Album Total Time' ,
              maximum_bin_size_in_hours = 10,
              maximum_bin_size_in_gb = 10)




get_pl_by_locations(
                    lib_name = lib_name, 
                    audio_locations = audio_locations, 
                    levels = levels)


NameError: name 'get_lib_as_csv' is not defined

In [ ]:
def assign_album_features_to_df(path_to_lib,
                        lib_version,
                       pl_list =['0','*', '**', '***', '****', '*****']
                       ):
    """_summary_

    Args:
        path_to_lib (_type_): _description_
        lib_version (_type_): _description_
        pl_list (list, optional): _description_. Defaults to ['*', '**', '*** 1', '**** 1', '***** 1'].

    Returns:
        _type_: _description_
    """
    
    df_lib = assign_score_to_df(path_to_lib,
                        lib_version,
                       pl_list =pl_list
                       )

    
    df_lib.loc[:,'Album'] =  df_lib['Album'].apply(lambda x: x[:-1] if x.endswith(' ') else x)
    df_lib.loc[:,'Album Artist'] =  df_lib['Album Artist'].apply(lambda x: str(x)[:-1] if str(x).endswith(' ') else str(x))
    
    
    group_features = ['Album', 
                        'Album Artist', 
                        'Disc Number',
                        'Genre',  
                        'Disc Count'
                        ]

    grouped_df = df_lib.groupby(by=group_features).apply(group_by_cd).reset_index()

    #grouped_df.loc[:,'cumsum_time'] = np.cumsum((grouped_df.loc[:,'Total Time']/1000)/(4*60*60))

    df_lib = df_lib.merge(grouped_df, on=group_features)

    return df_lib,grouped_df